In [1]:
import emojifi as emojifi
import glove as glove
from utils import convert_to_one_hot

/usr/lib/python3/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Load words and embeddings from pretrained Twitter gloVe

In [2]:
word_to_index, index_to_word, word_to_vec_map = glove.read_glove_vecs()

In [3]:
# Quick sanity check
print(list(word_to_index.items())[:10])
print("---")
print(list(index_to_word.items())[:10])

[('<user>', 0), ('.', 1), (':', 2), ('rt', 3), (',', 4), ('<repeat>', 5), ('<hashtag>', 6), ('<number>', 7), ('<url>', 8), ('!', 9)]
---
[(0, '<user>'), (1, '.'), (2, ':'), (3, 'rt'), (4, ','), (5, '<repeat>'), (6, '<hashtag>'), (7, '<number>'), (8, '<url>'), (9, '!')]


Load the gloVe embedding layer into Keras

In [4]:
embedding_layer = emojifi.pretrained_embedding_layer(word_to_vec_map, word_to_index)
print(embedding_layer.get_weights()[0][1])

[ 0.68661  -1.0772    0.011114 -0.24075  -0.3422    0.64456   0.54957
  0.30411  -0.54682   1.4695    0.43648  -0.34223  -2.7189    0.46021
  0.016881  0.13953   0.020913  0.050963 -0.48108  -1.0764   -0.16807
 -0.014315 -0.55055   0.67823   0.24359  -1.3179   -0.036348 -0.228
  1.0337   -0.53221  -0.52934   0.35537  -0.44911   0.79506   0.56947
  0.071642 -0.27455  -0.056911 -0.42961  -0.64412  -1.3495    0.23258
  0.25383  -0.10226   0.65824   0.16015   0.20959  -0.067516 -0.51952
 -0.34922 ]


In [5]:
max_word_count, X_train, Y_train, X_test, Y_test = emojifi.load_dataset()

In [19]:
#sanity check
print(max_word_count)

10


In [15]:
X_train_indices = emojifi.words_to_indices(X_train, word_to_index, max_word_count)
Y_train_oh = convert_to_one_hot(Y_train, C=5)
X_test_indices = emojifi.words_to_indices(X_test, word_to_index, max_word_count)
Y_test_oh = convert_to_one_hot(Y_test, C=5)

Create the Keras model

In [11]:
model = emojifi.load_model((max_word_count,), word_to_vec_map, word_to_index)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 10)                0         
_________________________________________________________________
embedding_3 (Embedding)      (None, 10, 50)            60000050  
_________________________________________________________________
lstm_3 (LSTM)                (None, 10, 128)           91648     
_________________________________________________________________
dropout_3 (Dropout)          (None, 10, 128)           0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dropout_4 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 5)                 645       
__________

Train the Keras model

NOTE: On 1070GTX this taxes ~8 seconds

NOTE: Train accuracy is currently 96+%

In [21]:
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
model.fit(X_train_indices, Y_train_oh, epochs = 50, batch_size = 32, shuffle=True)

Epoch 1/50
132/132 [==============================] - 1s 9ms/step - loss: 0.9428 - acc: 0.9621
Epoch 2/50
132/132 [==============================] - 0s 704us/step - loss: 0.9452 - acc: 0.9621
Epoch 3/50
132/132 [==============================] - 0s 740us/step - loss: 0.9638 - acc: 0.9394
Epoch 4/50
132/132 [==============================] - 0s 744us/step - loss: 0.9499 - acc: 0.9545
Epoch 5/50
132/132 [==============================] - 0s 746us/step - loss: 0.9431 - acc: 0.9621
Epoch 6/50
132/132 [==============================] - 0s 751us/step - loss: 0.9594 - acc: 0.9470
Epoch 7/50
132/132 [==============================] - 0s 741us/step - loss: 0.9539 - acc: 0.9470
Epoch 8/50
132/132 [==============================] - 0s 745us/step - loss: 1.0241 - acc: 0.8864
Epoch 9/50
132/132 [==============================] - 0s 753us/step - loss: 0.9719 - acc: 0.9318
Epoch 10/50
132/132 [==============================] - 0s 758us/step - loss: 1.0274 - acc: 0.8788
Epoch 11/50
132/132 [==========

NOTE: Test accuracy is falling between 75% and 85%

In [22]:
loss, accuracy = model.evaluate(X_test_indices, Y_test_oh)
print("\nTest accuracy = ", accuracy)

56/56 [==============================] - 0s 5ms/step

Test accuracy =  0.8035714200564793
